In [ ]:
import wikipedia
base_url = "/home/pushpa/Documents/Project_SRC_Results/SourceFiles/"

In [ ]:
from openie import StanfordOpenIE

with StanfordOpenIE() as client:
    text = 'With the increase of films released in 3D, 2D to 3D conversion has become more common. The majority of non-CGI stereo 3D blockbusters are converted fully or at least partially from 2D footage. Even Avatar contains several scenes shot in 2D and converted to stereo in post-production. The reasons for shooting in 2D instead of stereo are financial, technical and sometimes artistic:\
Stereo post-production workflow is much more complex and not as well-established as 2D workflow, requiring more work and rendering.\
Professional stereoscopic rigs are much more expensive and bulky than customary monocular cameras. Some shots, particularly action scenes, can be only shot with relatively small 2D cameras.\
Stereo cameras can introduce various mismatches in stereo image (such as vertical parallax, tilt, color shift, reflections and glares in different positions) that should be fixed in post-production anyway because they ruin the 3D effect. This correction sometimes may have complexity comparable to stereo conversion.\
Stereo cameras can betray practical effects used during filming.  For example, some scenes in the Lord of the Rings film trilogy were filmed using forced perspective to allow two actors to appear to be different physical sizes.  The same scene filmed in stereo would reveal that the actors were not the same distance from the camera.\
By their very nature, stereo cameras have restrictions on how far the camera can be from the filmed subject and still provide acceptable stereo separation.  For example, the simplest way to film a scene set on the side of a building might be to use a camera rig from across the street on a neighboring building, using a zoom lens.  However, while the zoom lens would provide acceptable image quality, the stereo separation would be virtually nil over such a distance.Even in the case of stereo shooting, conversion can frequently be necessary. Besides the mentioned hard-to-shoot scenes, there are situations when mismatches in stereo views are too big to adjust, and it is simpler to perform 2D to stereo conversion, treating one of the views as the original 2D source.'
    print('Text: %s.' % text)
    for triple in client.annotate(text):
        print('|-', triple)

    graph_image = 'graph.png'
    #client.generate_graphviz_graph(text, graph_image)
    #print('Graph generated: %s.' % graph_image)

    with open('/home/pushpa/Documents/Project_SRC_Results/SourceFiles/Film/2D to 3D conversion.txt', 'r', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')

    triples_corpus = client.annotate(corpus[0:50000])
    print('Corpus: %s [...].' % corpus[0:80])
    print('Found %s triples in the corpus.' % len(triples_corpus))
    for triple in triples_corpus[:3]:
        print('|-', triple)

In [ ]:
!export CORENLP_HOME="/home/pushpa/Documents/stanford-corenlp-latest/stanford-corenlp-4.2.0/"

In [ ]:
import os
os.environ["CORENLP_HOME"] = r'/home/pushpa/Documents/stanford-corenlp-latest/stanford-corenlp-4.2.0/'

In [ ]:
from stanfordnlp.server import CoreNLPClient
with open('/home/pushpa/Documents/Project_SRC_Results/SourceFiles/Film/2000s in film.txt', 'r', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')
text = corpus
print(f"Input text: {text}")

# set up the client

client = CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'}, timeout=90000, memory='16G')

# submit the request to the server
ann = client.annotate(text)    

mychains = list()
chains = ann.corefChain
for chain in chains:
    mychain = list()
    # Loop through every mention of this chain
    for mention in chain.mention:
        # Get the sentence in which this mention is located, and get the words which are part of this mention
        # (we can have more than one word, for example, a mention can be a pronoun like "he", but also a compound noun like "His wife Michelle")
        words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
        #build a string out of the words of this mention
        ment_word = ' '.join([x.word for x in words_list])
        mychain.append(ment_word)
    mychains.append(mychain)

for chain in mychains:
    print(' <-> '.join(chain))

In [ ]:
mychains

In [ ]:
!java -Xmx5g edu.stanford.nlp.pipeline.StanfordCoreNLP -file "2D to 3D conversion.txt"

In [2]:
import wikipediaapi  # pip install wikipedia-api
import pandas as pd
import concurrent.futures
from tqdm import tqdm

def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                return {'page': link, 'text': page.text, 'link': page.fullurl,
                        'categories': list(page.categories.keys())}
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        print('Page {} does not exist.'.format(topic_name))
        return
    
    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            sources.append(data) if data else None
            progress.update(1) if verbose else None     
    progress.close() if verbose else None
    
    namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                  'Template', 'Help', 'User', 'Category talk', 'Portal talk')
    sources = pd.DataFrame(sources)
    sources.to_csv(topic_name+"wiki.csv")
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(namespaces, na=True))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    print('Wikipedia pages scraped:', len(sources))
    
    return sources

In [4]:
#wiki_data = wiki_scrape('Film')

In [14]:
import pandas as pd
import re
import spacy
import neuralcoref
nlp = spacy.load('/home/pushpa/.local/lib/python3.6/site-packages/en_core_web_lg/en_core_web_lg-2.1.0')

neuralcoref.add_to_pipe(nlp)


def get_entity_pairs(text, coref=True):
    # preprocess text
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    if coref:
        text = nlp(text._.coref_resolved)  # resolve coreference clusters

    def refine_ent(ent, sent):
        unwanted_tokens = (
            'PRON',  # pronouns
            'PART',  # particle
            'DET',  # determiner
            'SCONJ',  # subordinating conjunction
            'PUNCT',  # punctuation
            'SYM',  # symbol
            'X',  # other
        )
        ent_type = ent.ent_type_  # get entity type
        if ent_type == '':
            ent_type = 'NOUN_CHUNK'
            ent = ' '.join(str(t.text) for t in
                           nlp(str(ent)) if t.pos_
                           not in unwanted_tokens and t.is_stop == False)
        elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
            refined = ''
            for i in range(len(sent) - ent.i):
                if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                    refined += ' ' + str(ent.nbor(i))
                else:
                    ent = refined.strip()
                    break

        return ent, ent_type

    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = []
    for sent in sentences:
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        #spans = spacy.util.filter_spans(spans)
        spans = filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span, attrs={'tag': span.root.tag,
                                            'dep': span.root.dep}) for span in spans]
        deps = [token.dep_ for token in sent]

        # limit our example to simple sentences with one subject and object
        if (deps.count('obj') + deps.count('dobj')) != 1\
                or (deps.count('subj') + deps.count('nsubj')) != 1:
            continue

        for token in sent:
            if token.dep_ not in ('obj', 'dobj'):  # identify object nodes
                continue
            subject = [w for w in token.head.lefts if w.dep_
                       in ('subj', 'nsubj')]  # identify subject nodes
            if subject:
                subject = subject[0]
                # identify relationship by root dependency
                relation = [w for w in token.ancestors if w.dep_ == 'ROOT']
                if relation:
                    relation = relation[0]
                    # add adposition or particle to relationship
                    if relation.nbor(1).pos_ in ('ADP', 'PART'):
                        relation = ' '.join((str(relation), str(relation.nbor(1))))
                else:
                    relation = 'unknown'

                subject, subject_type = refine_ent(subject, sent)
                token, object_type = refine_ent(token, sent)

                ent_pairs.append([str(subject), str(relation), str(token),
                                  str(subject_type), str(object_type)])

    ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(ent) == '' for ent in sublist)]
    pairs = pd.DataFrame(ent_pairs, columns=['subject', 'relation', 'object',
                                             'subject_type', 'object_type'])
    print('Entity pairs extracted:', str(len(ent_pairs)))

    return pairs


def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    # For spaCy 2.1.4+: this function is available as spacy.util.filter_spans()
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        # Check for end - 1 here because boundaries are inclusive
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
        seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result


In [3]:
import pandas as pd
wiki_data = pd.read_csv("processed_text_dataframe.csv")

In [4]:
wiki_data.columns=["No","text"]

In [8]:
wiki_data.iloc[0]["text"][0:1000]

'Mary Frances Reynolds (April 1, 1932 – December 28, 2016), known professionally as Debbie Reynolds, was an American actress, singer, and businesswoman. Her career spanned almost 70 years. She was nominated for the Golden Globe Award for Most Promising Newcomer for her portrayal of Helen Kane in the 1950 film Three Little Words, and her breakout role was her first leading role, as Kathy Selden in Singin\' in the Rain (1952). Her other successes include The Affairs of Dobie Gillis (1953), Susan Slept Here (1954), Bundle of Joy (1956 Golden Globe nomination), The Catered Affair (1956 National Board of Review Best Supporting Actress Winner), and Tammy and the Bachelor (1957), in which her performance of the song "Tammy" reached number one on the Billboard music charts. In 1959, she released her first pop music album, titled Debbie.She starred in Singin\' in the Rain (1952), How the West Was Won (1962), and The Unsinkable Molly Brown (1964), a biographical film about the famously boisterou

In [15]:
pairs = get_entity_pairs(wiki_data.iloc[0]["text"])

Entity pairs extracted: 31


In [ ]:
#!pip3 install spacy==2.1.4

In [16]:
pairs

,subject,relation,object,subject_type,object_type
0,American actress,released,American actress,NOUN_CHUNK,NOUN_CHUNK
1,American actress,include,The Singing Nun,NOUN_CHUNK,WORK_OF_ART
2,Reynolds,reached,new younger generation,PERSON,NOUN_CHUNK
3,American actress,released,American actress,NOUN_CHUNK,NOUN_CHUNK
4,American actress,released,second autobiography,NOUN_CHUNK,NOUN_CHUNK
5,Reynolds,had,business ventures,PERSON,NOUN_CHUNK
6,Reynolds,received,the Screen Actors Guild Life Achievement Award,PERSON,ORG
7,American actress,received,the Screen Actors Guild Life Achievement Award,NOUN_CHUNK,ORG
8,Debbie,has,refreshing sense,PERSON,NOUN_CHUNK
9,Debbie,led to,role,PERSON,NOUN_CHUNK


In [ ]:
doc = nlp(wiki_data.loc[0,'text'])
resolved_text = doc._.coref_resolved
sentences = [sent.string.strip() for sent in nlp(resolved_text).sents]
output = [sent for sent in sentences if 'president' in 
          (' '.join([token.lemma_.lower() for token in nlp(sent)]))]
print('Fact count:', len(output))
for fact in range(len(output)):
    print(str(fact+1)+'.', output[fact])

In [ ]:
sentences

In [ ]:
with open('/home/pushpa/Documents/Project_SRC_Results/SourceFiles/Film/2000s in film.txt', 'r', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')
text = corpus
doc = nlp(text)
resolved_text = doc._.coref_resolved
sentences = [sent.string.strip() for sent in nlp(resolved_text).sents]
output = [sent for sent in sentences if 'president' in 
          (' '.join([token.lemma_.lower() for token in nlp(sent)]))]
print('Fact count:', len(output))
for fact in range(len(output)):
    print(str(fact+1)+'.', output[fact])

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


def draw_kg(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
    node_deg = nx.degree(k_graph)
    layout = nx.spring_layout(k_graph, k=0.15, iterations=20)
    plt.figure(num=None, figsize=(100, 70), dpi=70)
    nx.draw_networkx(
        k_graph,
        arrowsize=7,
        linewidths=6.0,
        node_size=70, width=5.3, font_size =53,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white',
        )
    labels = dict(zip(list(zip(pairs.subject, pairs.object)),
                  pairs['relation'].tolist()))
    nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,
                                 font_color='red')
    plt.axis('off')
    plt.show()

In [ ]:
draw_kg(pairs)

In [ ]:
def filter_graph(pairs, node):
    k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
    edges = nx.dfs_successors(k_graph, node)
    nodes = []
    for k, v in edges.items():
        nodes.extend([k])
        nodes.extend(v)
    subgraph = k_graph.subgraph(nodes)
    layout = (nx.random_layout(k_graph))
    nx.draw_networkx(
        subgraph,
        node_size=1000,
        arrowsize=20,
        linewidths=1.5,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white'
        )
    labels = dict(zip((list(zip(pairs.subject, pairs.object))),
                    pairs['relation'].tolist()))
    edges= tuple(subgraph.out_edges(data=False))
    sublabels ={k: labels[k] for k in edges}
    nx.draw_networkx_edge_labels(subgraph, pos=layout, edge_labels=sublabels,
                                font_color='red')
    plt.axis('off')
    plt.show()

In [ ]:
filter_graph(pairs, 'American actress')

In [5]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")


Did not use initialization regex that was passed: _context_layer._module.weight_hh.*
Did not use initialization regex that was passed: _context_layer._module.weight_ih.*


In [6]:
wiki_data.iloc[3]["text"][0:500]

'The National Student Film Institute (NSFI), formerly the Los Angeles Student Film Institute (LASFI), was founded in 1978 by Brenda Norman, Dave Master, Jutti Marsh and Ralph Rogers as a festival for films made by children from kindergarten through ninth grade.  Two years later it was expanded to include the work of all high school students.  The first of what became an annual festival included approximately 350 students who entered 125 films.  By 1993, the Student Film Festival involved over two'

In [7]:
predictor.predict(
    document=wiki_data.iloc[3]["text"][0:500]
)

{'top_spans': [[0, 7],
  [0, 18],
  [0, 19],
  [10, 18],
  [11, 17],
  [11, 18],
  [17, 17],
  [21, 21],
  [23, 23],
  [25, 26],
  [25, 35],
  [28, 29],
  [28, 30],
  [28, 32],
  [31, 32],
  [34, 35],
  [37, 48],
  [40, 48],
  [43, 48],
  [50, 50],
  [51, 52],
  [51, 53],
  [54, 54],
  [56, 56],
  [58, 58],
  [59, 65],
  [62, 65],
  [67, 67],
  [68, 75],
  [73, 75],
  [76, 76],
  [77, 83],
  [81, 81],
  [82, 83],
  [85, 85],
  [87, 87],
  [89, 92],
  [93, 93]],
 'antecedent_indices': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37]

In [ ]:
#!pip install allennlp==1.0.0 allennlp-models==1.0.0

In [7]:
sent = wiki_data.iloc[3]["text"][0:500]

pred = predictor.predict(document= sent)
clusters = pred['clusters']
document = pred['document'] 

n = 0
doc = {}
for obj in document:    
    doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.    
    n = n+1

In [ ]:
clus_all = []
cluster = []
clus_one = {}
for i in range(0, len(clusters)):    
    one_cl = clusters[i]    
    for count in range(0, len(one_cl)):           
        obj = one_cl[count]        
        for num in range((obj[0]), (obj[1]+1)):            
            for n in doc:                
                if num == n:                 
                    cluster.append(doc[n]) 
    clus_all.append(cluster)       
    cluster = []     
    print (clus_all) #And finally, this shows all coreferences

In [ ]:
!pip install nested-coref-resolver

In [9]:
from ncr.replace_corefs import resolve

In [10]:
example = {'document':document,'clusters':clusters}

In [11]:
resolved_toks = resolve(example['document'], example['clusters'])
print(' '.join(resolved_toks))

The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) , was founded in 1978 by Brenda Norman , Dave Master , Jutti Marsh and Ralph Rogers as a festival for films made by children from kindergarten through ninth grade .   Two years later The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) was expanded to include the work of all high school students .   The first of what became an annual festival included approximately 350 students who entered 125 films .   By 1993 , The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) involved over two


In [12]:
coref_resol=pd.DataFrame(columns=["coref_text"])

In [13]:
wiki_data= wiki_data[wiki_data["text"]!="None"]

In [14]:
for index, row in wiki_data.iterrows():
    print("row",index)
    if len(row["text"]) > 1001 : 
        sent = row["text"][0:1000]
    else :
        sent = row["text"]
    pred = predictor.predict(document= sent)
    clusters = pred['clusters']
    document = pred['document']
    example = {'document':document,'clusters':clusters}
    resolved_toks = resolve(example['document'], example['clusters'])
    df = pd.DataFrame([' '.join(resolved_toks)])
    coref_resol.append(df)
        

row 0
row 1
row 2
row 3
row 4
row 5
row 6
row 7
row 8
row 9
row 10
row 11
row 12
row 13
row 14
row 15
row 16
row 17
row 18
row 20
row 21
row 22
row 23
row 24
row 25
row 26
row 27
row 28
row 29
row 30
row 31
row 32
row 33
row 34
row 35
row 36
row 37
row 38
row 39
row 40
row 41
row 42
row 43
row 44
row 45
row 46
row 47
row 48
row 49
row 50
row 51
row 52
row 53
row 54
row 55
row 56
row 57
row 58
row 59
row 60
row 61
row 62
row 63
row 64
row 65
row 66
row 67
row 68
row 69
row 70
row 71
row 72
row 73
row 74
row 75
row 76
row 77
row 78
row 79
row 80
row 81
row 82
row 83
row 84
row 85
row 86
row 87
row 88
row 89
row 90
row 91
row 92
row 93
row 94
row 95
row 96
row 97
row 98
row 99
row 100
row 101
row 102
row 103
row 104
row 105
row 106
row 107
row 108
row 109
row 110
row 111
row 112
row 113
row 114
row 115
row 116
row 117
row 118
row 119
row 120
row 121
row 122
row 123
row 124
row 125
row 126
row 127
row 128
row 129
row 130
row 131
row 132
row 133
row 134
row 135
row 136
row 137
row 138
row 1

In [ ]:
##############open ie ###########################
from openie import StanfordOpenIE

with StanfordOpenIE() as client:
    text = 'The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) , was founded in 1978 by Brenda Norman , Dave Master , Jutti Marsh and Ralph Rogers as a festival for films made by children from kindergarten through ninth grade .   Two years later The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) was expanded to include the work of all high school students .   The first of what became an annual festival included approximately 350 students who entered 125 films .   By 1993 , The National Student Film Institute ( NSFI ) , formerly the Los Angeles Student Film Institute ( LASFI ) involved over two'
    print('Text: %s.' % text)
    for triple in client.annotate(text):
        print('|-', triple)

    graph_image = 'graph.png'
    #client.generate_graphviz_graph(text, graph_image)
    #print('Graph generated: %s.' % graph_image)

    with open('/home/pushpa/Documents/Project_SRC_Results/SourceFiles/Film/2D to 3D conversion.txt', 'r', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')

    triples_corpus = client.annotate(corpus[0:50000])
    print('Corpus: %s [...].' % corpus[0:80])
    print('Found %s triples in the corpus.' % len(triples_corpus))
    for triple in triples_corpus[:3]:
        print('|-', triple)

In [1]:
coref_resol

NameError: name 'coref_resol' is not defined

In [32]:
wiki_data

,No,text
0,0,"Mary Frances Reynolds (April 1, 1932 – Decembe..."
1,1,A digital copy is a commercially distributed c...
2,2,"This is a list of films from Blue Sky Studios,..."
3,3,"The National Student Film Institute (NSFI), fo..."
4,4,A film studio (also known as movie studio or s...
...,...,...
488,488,2D to 3D video conversion (also called 2D to s...
489,489,Film Sack is a weekly podcast focused on film ...
490,490,The film industry in Florida is one of the lar...
491,491,Films that feature body swapping as a prominen...
